In [1]:
#!/usr/bin/env python3
"""
edge_with_coords.py
Menampilkan hasil deteksi tepi (Canny) dan koordinat setiap titik sudut dokumen (kontur 4 titik).

Fungsi:
- Membaca semua gambar dari data_mentah/
- Melakukan deteksi tepi (Canny)
- Menemukan semua kontur quadrilateral (4 titik)
- Menggambar titik-titik sudut dan menampilkan koordinatnya
- Menyimpan hasilnya sebagai gambar hitam putih di data_output/

Tidak ada hasil scan atau transformasi perspektif.
"""

import cv2
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

# Folder input dan output
INPUT_FOLDER = Path("data_mentah")
OUTPUT_FOLDER = Path("data_output")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

# Parameter deteksi
MIN_CONTOUR_AREA = 1000
APPROX_EPSILON_RATIO = 0.02

# Warna titik dan garis (BGR)
COLOR_POINT = (0, 0, 255)   # merah
COLOR_LINE = (0, 255, 0)    # hijau

def urutkan_titik(pts):
    """Urutkan titik menjadi [tl, tr, br, bl]."""
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]   # top-left
    rect[2] = pts[np.argmax(s)]   # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect

def detect_edges_and_coords(image_path):
    """Deteksi tepi dan tampilkan koordinat titik kontur 4 titik."""
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"❌ Gagal membaca {image_path}")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)

    # Deteksi tepi (Canny)
    edged = cv2.Canny(gray, 50, 150)

    # Temukan kontur
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    quads = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < MIN_CONTOUR_AREA:
            continue
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, APPROX_EPSILON_RATIO * peri, True)
        if len(approx) == 4:
            quads.append(approx.reshape(4, 2).astype("float32"))

    # Buat gambar hitam putih untuk anotasi
    edge_bgr = cv2.cvtColor(edged, cv2.COLOR_GRAY2BGR)

    pil_img = Image.fromarray(cv2.cvtColor(edge_bgr, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 14)
    except:
        font = ImageFont.load_default()

    print(f"\n🖼 {image_path.name} — ditemukan {len(quads)} kontur quadrilateral")
    for idx, quad in enumerate(quads, start=1):
        rect = urutkan_titik(quad)
        color = (255, 0, 0)
        pts = rect.astype(int).tolist()

        # Gambar garis hijau
        for i in range(4):
            pt1 = tuple(pts[i])
            pt2 = tuple(pts[(i+1)%4])
            cv2.line(edge_bgr, pt1, pt2, COLOR_LINE, 2)

        # Gambar titik dan teks koordinat
        for i, (x, y) in enumerate(pts):
            cv2.circle(edge_bgr, (x, y), 5, COLOR_POINT, -1)
            label = f"{idx}-{['TL','TR','BR','BL'][i]} ({x},{y})"
            draw.text((x + 10, y - 10), label, fill=(255,255,255), font=font)
            draw.text((x + 10, y - 10), label, fill=(0,0,0), font=font)

    # Simpan hasil ke data_output
    out_file = OUTPUT_FOLDER / f"{image_path.stem}_edges_with_coords.jpg"
    final_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    cv2.imwrite(str(out_file), final_img)
    print(f"   ✅ Disimpan ke: {out_file}")

def process_all_images():
    files = list(INPUT_FOLDER.glob("*.jpg")) + list(INPUT_FOLDER.glob("*.png")) + list(INPUT_FOLDER.glob("*.jpeg"))
    if not files:
        print("⚠️  Tidak ada gambar di folder data_mentah/")
        return

    for f in files:
        detect_edges_and_coords(f)

    print(f"\n📂 Semua hasil disimpan di: {OUTPUT_FOLDER.resolve()}")

if __name__ == "__main__":
    process_all_images()



🖼 data_a1.jpg — ditemukan 524 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a1_edges_with_coords.jpg

🖼 data_a10.jpg — ditemukan 419 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a10_edges_with_coords.jpg

🖼 data_a100.jpg — ditemukan 290 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a100_edges_with_coords.jpg

🖼 data_a101.jpg — ditemukan 324 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a101_edges_with_coords.jpg

🖼 data_a102.jpg — ditemukan 404 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a102_edges_with_coords.jpg

🖼 data_a103.jpg — ditemukan 347 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a103_edges_with_coords.jpg

🖼 data_a104.jpg — ditemukan 392 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a104_edges_with_coords.jpg

🖼 data_a105.jpg — ditemukan 353 kontur quadrilateral
   ✅ Disimpan ke: data_output\data_a105_edges_with_coords.jpg

🖼 data_a107.jpg — ditemukan 360 kontur quadrilateral
   ✅ Disimpan ke: data_o